In [2]:
from keras import Sequential
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, UpSampling2D
from keras.utils import to_categorical
from keras.datasets import cifar10
import pandas as pd
import pickle 
import numpy as np
import matplotlib.pyplot as plt
def resnet_model(froze=False, LR_scheduler='Plateau', augmentation=True, batch_size=100, epochs=30, learn_rate=.001):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = preprocess_input(x_train)
    x_test = preprocess_input(x_test)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    y_test = to_categorical(y_test)

    train_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)

    val_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)

    test_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)
    train_generator.fit(x_train)
    val_generator.fit(x_val)
    test_generator.fit(x_test) 
    
    # setting learning rate schedulers
    def scheduler(epoch):
        if epoch < 5:
            return learn_rate 
        if epoch < 20:
            return learn_rate /10
        return learn_rate/100
    if LR_scheduler == 'Custom' :
        callbacks = [LearningRateScheduler(scheduler)]
    elif LR_scheduler == 'Plateau' :
        callbacks = [ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=.01,
        patience=3,
        min_lr=1e-5)]
    else :
        callbacks = [LearningRateScheduler(lambda x: learn_rate)]
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3),
                          classes=y_train.shape[1])

    callbacks.append(EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01))
    model = Sequential()
    model.add(UpSampling2D(size=(2,2))) 
    model.add(UpSampling2D(size=(2,2))) 
    model.add(base_model)
    model.add(Flatten())
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.layers[3].trainable = not froze

    adam = Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    if augmentation:
        history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size),
                  epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                  validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps=10,
                  callbacks=callbacks, verbose=1)
    else:
        history = model.fit(x_train, y_train, batch_size=batch_size,
                  epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                  validation_data=(x_val, y_val), validation_steps=10,
                  callbacks=callbacks, verbose=1)
    return model



In [ ]:
model = resnet_model(LR_scheduler='Custom')
model

94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/30
350/350 [==============================] - 76s 194ms/step - loss: 1.5574 - accuracy: 0.4727 - val_loss: 1.0745 - val_accuracy: 0.6550
Epoch 2/30
350/350 [==============================] - 66s 189ms/step - loss: 0.7452 - accuracy: 0.7634 - val_loss: 0.9652 - val_accuracy: 0.6960
Epoch 3/30
350/350 [==============================] - 66s 190ms/step - loss: 0.6283 - accuracy: 0.7996 - val_loss: 0.9186 - val_accuracy: 0.7180
Epoch 4/30
350/350 [==============================] - 66s 189ms/step - loss: 0.5330 - accuracy: 0.8299 - val_loss: 0.7168 - val_accuracy: 0.7840
Epoch 5/30
350/350 [==============================] - 66s 189ms/step - loss: 0.4498 - accuracy: 0.8554 - val_loss: 0.7306 - val_accuracy: 0.7460
Epoch 6/30
350/350 [==============================] - 66s 189ms/step - loss: 0.3592 - accuracy: 0.8861 - val_loss: 0.3437 - val_accuracy: 0.8890
Epoch 7/30
337/350 [===========================>..] - ETA: 2s - l

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)
res = model.predict(x_test)
(res.argmax(axis=-1) == y_test.argmax(axis = 1)).mean()

In [ ]:
!pip install py7zr
import py7zr

archive = py7zr.SevenZipFile('../input/cifar-10/test.7z', mode='r')
archive.extractall(path="test1")
archive.close()


In [ ]:

images = np.array([plt.imread(f'./test1/test/{i+1}.png') for i in range(300000)])

In [ ]:
images.shape


In [ ]:
res = model.predict(preprocess_input(images))

In [ ]:
pd.DataFrame(res, columns = range(10)).to_csv('probab.csv', index=False)

In [ ]:
kaggle = pd.DataFrame(res.argmax(axis=-1)).reset_index()
kaggle['index'] = kaggle['index']+1
kaggle

In [ ]:
kaggle.columns = ['id' , 'label']

In [ ]:
mapping = {'0':'airplane', '1':'automobile', '2':'bird', '3':'cat', '4':'deer', '5':'dog', '6':'frog', '7':'horse', '8':'ship', '9':'truck'}
kaggle['label'] = [mapping[str(i)] for i in kaggle['label']]
kaggle

In [ ]:
kaggle.to_csv('kaggle.csv', index=False)